### 原型DEMO
用途, 目前发现记录日志很好用

In [1]:
# singleton.py demo
class SingletonObject(object):
    class __SingletonObject():
        def __init__(self):
            self.val = None

        def __str__(self):
            return "{0!r} {1}".format(self, self.val)

    instance = None

    def __new__(cls):
        if not SingletonObject.instance:
            SingletonObject.instance = SingletonObject.__SingletonObject()

        return SingletonObject.instance

    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __setattr__(self, name):
        return setattr(self.instance, name)

###### 分析

嵌套类设计:  
外层SingletonObject是代理类  
内层__SingletonObject是实际实现类（双下划线表示私有类） 
通过instance类变量持有唯一实例  

关键方法： 
__new__：控制实例化过程，确保只创建一个实例  
__getattr__/__setattr__：将所有操作委托给内部实例  
__str__：定义实例的字符串表示形式  

实现特点：  
线程不安全的经典实现  
通过重写__new__方法实现单例  
使用属性委托模式  

最基础的语句, 打开文件写内容:
~~~python
with open("filename.log", "a") as log_file:
    log_file.write("Log message goes here.\n")
~~~

模块化
直接导入后使用, logger.log_message()
  
~~~python
# logger.py
def log_message(msg):
    with open("/var/log/filename.log", "a") as log_file:
        log_file.write("{0}\n".format(msg))
~~~

~~~python
# main.py
import logger

for i in range(4):
    logger.log_message("log message {}".format(i))
~~~

用类的方式重写一下,主要关注如何调用  

In [ ]:
# logger_class.py
class Logger(object):
    """A file-based message logger with the following properties
    
    Attributes:
        file_name: a string representing the full path of the log file to which
        this logger will write its messages
    """
    
    def __init__(self, file_name):
        """Return a Logger object whose file_name is *file_name*"""
        self.file_name = file_name
    
    def _write_log(self, level, msg):
        """Writes a message to the file_name for a specific Logger instance"""
        with open(self.file_name, "a") as log_file:
            log_file.write("[{0}] {1}\n".format(level, msg))
    
    def critical(self, level, msg):
        self._write_log("CRITICAL", msg)
    
    def error(self, level, msg):
        self._write_log("ERROR", msg)
    
    def warn(self, level, msg):
        self._write_log("WARN", msg)
    
    def info(self, level, msg):
        self._write_log("INFO", msg)
    
    def debug(self, level, msg):
        self._write_log("DEBUG", msg)

# from logger_class import Logger
logger_object = Logger("class_logger.log")
logger_object.info("Initialization", "Logger initialized successfully.")

实用化,改造成单例  
对比原型,只改了路径,然后做了接口调用,但日志等级没做,这是框架的事      

嵌套类结构：  
外层 SingletonObject 是代理类  
内层 __SingletonObject (双下划线表示私有类)是实际实现类  

核心机制：  
instance 类变量保存唯一实例  
__new__ 控制实例创建，确保只生成一个实例  
__getattr__/__setattr__ 将所有操作委托给内部实例  

属性存储：  
val 是存储在实例中的任意值  
__str__ 提供对象的字符串表示  

In [4]:
# singleton_object.py
class SingletonObject(object):
    class __SingletonObject():
        def __init__(self):
            # self.log_file = "myapp.log"  # 默认日志文件路径
            self.log_file = "/var/log/myapp.log"  # 默认日志文件路径
            
        def log(self, message):
            with open(self.log_file, "a") as f:
                f.write(f"{message}\n")
                
        def set_log_file(self, path):
            self.log_file = path

    instance = None

    def __new__(cls):
        if not SingletonObject.instance:
            SingletonObject.instance = SingletonObject.__SingletonObject()
        return SingletonObject.instance

    def __getattr__(self, name):
        return getattr(self.instance, name)

    def __setattr__(self, name, value):
        return setattr(self.instance, name, value)
    
# logger.py
# from singleton_object import SingletonObject

logger = SingletonObject()

def log_message(msg):
    logger.log(msg)
    
def set_log_path(path):
    logger.set_log_file(path)  

# from logger import log_message, set_log_path

# 可以自定义日志路径(可选)
set_log_path("my_custom.log")
# set_log_path("/var/log/my_custom.log")

# 记录日志
log_message("应用程序启动")
log_message("正在处理数据...")
log_message("操作完成")


# another_file.py
# from logger import log_message
log_message("从另一个模块记录日志")    